In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## Libraries and Filesystem Setup

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import pandas as pd
from collections import Counter
import pickle

[nltk_data] Downloading package punkt to /Users/dkarsann/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dkarsann/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Import BOW document vectors

In [3]:
BOW = pickle.load(open("../processing_files/bow.pickle", "rb"))
df_merged = pickle.load(open("../processing_files/df_merged.pickle", "rb"))

## similarity calculation

In [4]:
from sklearn.preprocessing import normalize
def cosine_sim(mat_name):
        norm_mat = normalize(mat_name, norm='l2', axis=1)
        cosine_mat = norm_mat @ norm_mat.T
        return cosine_mat.toarray()

In [5]:
cos_sim = cosine_sim(BOW)
cos_sim.shape

(12949, 12949)

## Similarity score dataframe

In [6]:
r_index = pd.Index(df_merged.DOMAIN.unique(), name='domain')
def similarity(row):
    idx = r_index.get_loc(row)
    sim = cos_sim[idx, :].copy()
    sim[idx] = 0
    domain_sim = pd.Series(sim, r_index, name='similarity')
    return domain_sim

In [7]:
df_en_sim = df_merged[['DOMAIN']]
domains = pd.Series(df_merged['DOMAIN'].unique())
scores = domains.apply(lambda x: similarity(x))
scores['DOMAIN'] = domains
#scores.to_parquet('count_similarity.parquet')
scores.head()

ValueError: Length of passed values is 12949, index implies 12946